In [1]:
import pandas as pd

In [103]:
indian_wells_2021 = "https://www.endurance-data.com/en/results/621-ironman-703-indian-wells-la-quinta/all/{results_page}/"
indian_wells_2019 = "https://www.endurance-data.com/en/results/477-ironman-703-indian-wells-la-quinta/all/{results_page}/"

In [114]:
race = indian_wells_2021


new_df_len = 1
results_page = 1
df = pd.DataFrame()

while new_df_len > 0:
    try:
        new_df = pd.read_html(race.format(results_page=results_page))[0]
        new_df_len = new_df.shape[0]
        if new_df_len > 0:
            df = pd.concat([df,new_df])
        results_page += 1
    except:
        new_df_len = 0

In [115]:
df.columns = ['overall_place','gender_place','division_place','name','bib','division','nationality','swim','bike','run','total']

In [121]:
def convert_time_seconds(time_str):
    hr_, min_, sec_ = time_str.split(':')
    total_seconds = int(hr_)*60*60 + int(min_)*60 + int(sec_)
    return total_seconds
def convert_seconds_time(seconds):
    seconds = int(seconds)
    hr_ = seconds // (60*60)
    min_ = (seconds - hr_*60*60) // 60
    sec_ = (seconds - hr_*60*60) - min_*60
    time_bits = [hr_,min_,sec_]
    for time_index in range(3):
        time_bits[time_index] = str(time_bits[time_index])
        time_bits[time_index] = '0' + time_bits[time_index] if len(time_bits[time_index]) < 2 else time_bits[time_index]
    time_str = ':'.join(time_bits)
    return time_str
def convert_sec_pace(seconds,discipline):
    if discipline == 'bike':
        dist = 90
    elif discipline == 'run':
        dist = 21.1
    elif discipline == 'swim':
        dist = 1.9
    else:
        return None
    return dist/seconds*60*60

In [117]:
for column in ['swim','bike','run','total']:
    df[f'{column}_seconds']  = df[column].map(convert_time_seconds)

# Division analysis

In [118]:
select_divs = ['M25-29','M30-34']
dfd = df[df['division'].isin(select_divs)]

## Total time in 5% steps of top 25%

In [119]:
quants = [.00001,.05,.1,.15,.2,.25]
dfd.quantile(quants)[['swim_seconds','bike_seconds','run_seconds']].sum(axis=1).map(convert_seconds_time)

0.00001    03:54:02
0.05000    04:23:01
0.10000    04:39:06
0.15000    04:54:02
0.20000    05:02:08
0.25000    05:10:35
dtype: object

## Discipline quantiles

In [128]:
for discipline in ['swim_seconds','bike_seconds','run_seconds']:
    d_q = dfd.quantile(quants)[discipline].map(convert_seconds_time)
    dp_q = dfd.quantile(quants)[discipline].apply(lambda x: convert_sec_pace(x,discipline.split('_')[0]))
    print(discipline.split('_')[0],'\n',pd.concat([d_q,dp_q],axis=1))

swim 
          swim_seconds  swim_seconds
0.00001      00:23:46      4.795523
0.05000      00:28:33      3.992179
0.10000      00:30:47      3.702501
0.15000      00:32:29      3.509222
0.20000      00:34:03      3.347035
0.25000      00:35:29      3.212399
bike 
          bike_seconds  bike_seconds
0.00001      02:11:30     41.062400
0.05000      02:22:25     37.916023
0.10000      02:29:02     36.230263
0.15000      02:35:35     34.707158
0.20000      02:38:42     34.025750
0.25000      02:41:33     33.426184
run 
          run_seconds  run_seconds
0.00001     01:18:46    16.072717
0.05000     01:32:02    13.753644
0.10000     01:39:16    12.752455
0.15000     01:45:58    11.946214
0.20000     01:49:22    11.575386
0.25000     01:53:33    11.148455
